In [ ]:
#Loading in Data 

with open('alljansom_p22mslp.p', 'rb') as infile:
    alljansommslp = pickle.load(infile)      #Here I am loading the data back in so I can use it to graph

alljanmslp = alljansommslp._weights   #You want the weight values here so you will assign the weights to a variable 

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pyplot import Normalize
import matplotlib as mpl


datacrs = ccrs.PlateCarree()   #Do not forget this line or the whoe thing will not work!


nrows = alljanmslp.shape[1]  
ncols = alljanmslp.shape[0]


fig, axs = plt.subplots(nrows=nrows,ncols=ncols,
                        subplot_kw={'projection':ccrs.LambertConformal(central_longitude=-156, central_latitude=71, standard_parallels=(30, 60))},
                        figsize=(30, 15),facecolor='white') 
#^That code is generating a plot that is positioned at the NSA site so you will need to adjust for the domain you want to use 
fig.tight_layout()


axs=axs.flatten()

for k in range(alljanmslp.shape[1]):
    for i in range(alljanmslp.shape[0]):
        datamslp = alljanmslp[i,k,0:5916].reshape(nx,ny)    #here I am going through the data to seperate the variables as remember they are all in one long row. You will need to remember the order of the variables.
        datauwind = alljanmslp[i,k,5916:11832].reshape(nx,ny) #you do not need to worry here about how they start at 0 so it is off by one! if the first data set goes to 5916 you do not need to put 0 to 5915!! that will break it 
        datavwind = alljanmslp[i,k,11832:17748].reshape(nx,ny)
        windspeed = (datauwind ** 2 + datavwind ** 2) ** 0.5

        
        node=(i+1)+k*alljanmslp.shape[0]   # the number of nodes you have depends on the x and y you selected in the SOM code block
        cs2=axs[i+k*alljanmslp.shape[0]].contour(lon, lat, datamslp,
                          transform = ccrs.PlateCarree(),
                          colors='black',extend='both')
        windspeed[windspeed < 5] = np.nan
        
        cs3=axs[i+k*alljanmslp.shape[0]].contourf(lon, lat, windspeed,
                        transform = ccrs.PlateCarree(),
                        cmap='cool', extend='both')
        
        cs4=axs[i+k*alljanmslp.shape[0]].quiver(lon, lat, datauwind, datavwind, pivot='middle', color='black',
                                                     regrid_shape=15 , headwidth= '10', headlength= '10',transform=datacrs)
        #^^^ this code block is plotting wind magnitude vectors

        axs[i+k*alljanmslp.shape[0]].set_extent([-131.7, -180, 73.48, 62.9], ccrs.PlateCarree() )
        #^You will need to adjust the axis extents for the region of study you have 
        
        axs[i+k*alljanmslp.shape[0]].coastlines()    #adding coastlines
        axs[i+k*alljanmslp.shape[0]].add_feature(cfeature.BORDERS)   #adding state borders
        axs[i+k*alljanmslp.shape[0]].scatter(-156.36,71.19, c='yellow',marker= 'o',s=120, linewidth=2,edgecolors= "black" ,zorder= 4,transform=datacrs)
        #^this code can mark a certain spot with a yellow circle. 
        

        # Title each subplot 
        axs[i+k*alljanmslp.shape[0]].set_title('Node:'+str(node), fontsize=18)
        #^this labels each plot with the correlating node!
     
        
        plt.tight_layout()
        fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.6,
                    wspace=0.05, hspace=0.25)

cbar_ax = fig.add_axes([0.08, 0.2, 0.5, 0.02])

cbar=fig.colorbar(cs3,cax=cbar_ax,orientation='horizontal')

cbar.set_label('Wind Speed (Knots)', fontsize=22)

plt.suptitle('SOM Nodes All Hours: MSLP Anomalies (hPa) Jan. 1990-2020 with Wind Magntiude Vectors (Knots) [Kennedy Parameters].', x= 0.33 ,fontsize=22) #adds a title
#plt.savefig('SOM Nodes all Hours: MSLP (hPa) Jan. 1990 [Kennedy Parameters].'.png')   #you can use this to save the plot!